In [ ]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization
from tensorflow.keras.layers import Conv3D, MaxPooling3D
from tensorflow.keras import backend as K
from tensorflow.keras import optimizers
from matplotlib import pyplot as plt
from IPython.display import clear_output
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.models import load_model
%matplotlib inline
# Load the TensorBoard notebook extension.
%load_ext tensorboard
#from keras_radam import RAdam
import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn as sk
import time
import sys
import collections
# Keras turner imports
import kerastuner
from kerastuner.tuners import RandomSearch, BayesianOptimization 
from kerastuner.engine.hyperparameters import HyperParameters
import os

In [ ]:
run_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\total_flux\\models"
data_directory = "Z:\\FNS_GA_v5_100_gen_CNN_hyperparam_work\\models\\total_flux\\data"
train_data_string = ["X_tf_times_source_training_data.csv.npy",
                     "Y_tf_times_source_training_data.csv.npy"]
validation_data_string = ["X_tf_times_source_validation_data.csv.npy",
                          "Y_tf_times_source_validation_data.csv.npy"]

In [ ]:
print(f"Tensor Flow Version: {tf.__version__}")
print(f"Keras Version: {tf.keras.__version__}")
print()
print(f"Python {sys.version}")
print(f"Pandas {pd.__version__}")
print(f"Scikit-Learn {sk.__version__}")
gpu = len(tf.config.list_physical_devices('GPU'))>0
print("GPU is", "available" if gpu else "NOT AVAILABLE")

In [ ]:
#data_ = np.loadtxt("FNS_GA_Data_Runs_11_12_Paper_2.csv", delimiter=",")
#X = data_[:,0:150]
os.chdir(data_directory)
X_train = np.load(train_data_string[0])
Y_train = np.load(train_data_string[1])


#data_test = np.loadtxt("FNS_GA_Run_13_Data_Test.csv", delimiter=",")
X_test = np.load(validation_data_string[0])
Y_test = np.load(validation_data_string[1])

#X_test = np.load("test_2x2_fns_run_test_dat.npy")

In [ ]:
from kerastuner import HyperModel
class HyperModel_555_v2(HyperModel):
    def __init__(self, input_shape):
        self.input_shape = input_shape
    def build(self, hp):
        conv_size = hp.Int("first_conv_size", min_value=16, max_value=64, step=1)
        kernel_size = hp.Int("kernel_size", min_value=4, max_value=10, step=1)
        dense_layer_size = hp.Int("dense_layer_size", min_value=32, max_value=128, step=32)
        max_pool_val = hp.Int("max_pool_val", min_value=2, max_value=10, step=1)

        model = Sequential()
        model.add(Conv3D(conv_size, kernel_size=(kernel_size, 2, 2),
                         input_shape=self.input_shape, padding='same', data_format="channels_last", strides = (1, 1, 1)))
        model.add(LeakyReLU(alpha=0.3))
        model.add(BatchNormalization())
        ##model.add(Conv3D(32, (3, 3, 5), activation=LeakyReLU(alpha=0.3), padding='same'))

        for i in range(hp.Int("n_layers", 7, 12)):
            model.add(Conv3D(hp.Int(f"conv_{i}_size",
                                    min_value=16,
                                    max_value=64,
                                    step=8),
                             (kernel_size, 2, 2),
                             padding='same'))
            model.add(LeakyReLU(alpha=0.3))
            model.add(BatchNormalization())
        model.add(MaxPooling3D(pool_size=(max_pool_val, 2, 2)))
        model.add(Flatten())
        model.add(BatchNormalization())
        model.add(Dense(dense_layer_size))
        model.add(LeakyReLU(alpha=0.3))
        model.add(BatchNormalization())
        model.add(Dense(dense_layer_size))
        model.add(LeakyReLU(alpha=0.3))
        model.add(Dropout(hp.Float('dropout_value', min_value = 0.0, max_value = 0.4, step=0.1)))
        
        model.add(Dense(1, activation='linear'))

        ### Building the optimizer object
        #sgd = optimizers.SGD(lr = 0.1) # 1.0 0.0001 0.01
        opt = optimizers.Adam(lr = hp.Float(
                    'learning_rate',
                    min_value=0.0001,
                    max_value=0.04,
                    default=0.0001,
                    step=1e-4)) 
           
        ### Adding the loss function and optimizer to the model
        #mcp_save = ModelCheckpoint('CNN_3d_2x2_test_2.hdf5', save_best_only=True, monitor='val_loss', mode='min')
        #opt = RAdam(total_steps=100, warmup_proportion=0.1, min_lr=1e-5)
        model.compile(loss='mse', optimizer=opt, metrics=['mae'])
        ### Running the model


        return model

In [ ]:
input_shape = (60, 2, 2, 4)
hypermodel = HyperModel_555_v2(input_shape)

In [ ]:
os.chdir(run_directory)
LOG_DIR = "bayes_tf_times_source_1_"
max_trials_val = 100
# int(max_trials_val/10)
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_mae',
            max_trials=max_trials_val,
            seed=1,
            executions_per_trial=2,
            num_initial_points = int(max_trials_val/10),
            directory = LOG_DIR)
##tuner_bo.search(x=X_train, y=Y_train,
##             epochs=300,
##             batch_size=2000,
##             validation_data=(X_test, Y_test))

In [ ]:
#tuner_bo.results_summary()

In [ ]:
#best_model = tuner_bo.get_best_models(num_models=10)
#best_model[0].save('best_model_bayes_test_keff_gen_46_2')

In [ ]:
def train_top_models(models_list,
                     model_string_template = "test",
                     csv_logger_bool = True,
                     model_checkpoint_boo = True,
                     save_original_model = True,
                     train_models = True,
                     train_model_options = {}):
    
    for model_count, current_model in enumerate(models_list):
        model_string = model_string_template + "_model_" + str(model_count)
        
        callback_list = []
        if csv_logger_bool:
            csv_logger = CSVLogger(model_string + '.log')
            callback_list.append(csv_logger)
            
        if model_checkpoint_boo:
            checkpoint_filepath = model_string + "_chkpnt_.{epoch:02d}-{val_mae:.6f}.hdf5"
            model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                            filepath=checkpoint_filepath,
                                            monitor='val_mae',
                                            mode='min',
                                            save_best_only=True,
                                            verbose = 1)
            callback_list.append(model_checkpoint_callback)
            
        if save_original_model:
            current_model.save(model_string + '_original')
            
        if train_models:
            current_model.fit(train_model_options['x_train'],
                 train_model_options['y_train'],
                 validation_data=train_model_options['validation_data'],
                 epochs=train_model_options['epochs'],
                 batch_size=train_model_options['batch_size'],
                 verbose=train_model_options['verbose'],
                 callbacks=callback_list)

In [ ]:
best_models = tuner_bo.get_best_models(num_models=10)
train_model_opt = collections.OrderedDict()
train_model_opt['x_train'] = X_train
train_model_opt['y_train'] = Y_train
train_model_opt['validation_data'] = (X_test, Y_test)
train_model_opt['epochs'] = 1000
train_model_opt['batch_size'] = int(len(X_train)/2)
train_model_opt['verbose'] = 1


train_top_models(best_models, train_model_options = train_model_opt,model_string_template = "tf")

In [ ]:
loaded_model = keras.models.load_model("tf_model_1_chkpnt_.1737-1925532.000000.hdf5")
print(loaded_model.summary())
output_ = loaded_model.predict(X_test)
for real, pred in zip(Y_test, output_):
    print(real[0], pred[0])

In [ ]:

train_model_opt = collections.OrderedDict()
train_model_opt['x_train'] = X_train
train_model_opt['y_train'] = Y_train
train_model_opt['validation_data'] = (X_test, Y_test)
train_model_opt['epochs'] = 3000
train_model_opt['batch_size'] = int(len(X_train)/4)
train_model_opt['verbose'] = 1


train_top_models([loaded_model], model_string_template = "model_6_tf", train_model_options = train_model_opt)

In [ ]:
loaded_model = keras.models.load_model("model_6_tf_model_0_chkpnt_.707-0.000382.hdf5")
print(loaded_model.summary())
output_ = loaded_model.predict(X_test)
for real, pred in zip(Y_test, output_):
    print(real[0], pred[0])

In [ ]:
import glob
import os

list_of_files = glob.glob('*.hdf5') # * means all if need specific format then *.csv
real_list = []
for file in list_of_files:
    if "model_7" in file:
        real_list.append(file)
latest_file = max(real_list, key=os.path.getctime)
print(latest_file)

In [ ]:
def get_latest_files(file_flag, X_data, Y_data, file_count_value = 10, file_suffix = '.hdf5'):
    
    list_of_files = glob.glob('*.hdf5') # * means all if need specific format then *.csv
    real_list = []
    list_of_best_models = []
        
    for file_count in range(file_count_value):
        file_flag_ = file_flag + str(file_count)
        #print(file_flag_)
        for file in list_of_files:
            if file_flag_ in file:
                real_list.append(file)
        latest_file = max(real_list, key=os.path.getctime)    
        #print(latest_file)
        list_of_best_models.append(latest_file)
        
    pred_data = []
    for model in list_of_best_models:
        loaded_model = keras.models.load_model(model)
        pred_data.append(loaded_model.predict(X_data))
        
    for count, _ in enumerate(pred_data[0]):
        print_str = ''
        for count_model, __ in enumerate(pred_data):
            #print(pred_data[count_model][0][0])
            print_str += str(pred_data[count_model][count][0]) + "," 
        print(str(Y_data[count][0]) + "," + print_str)

In [ ]:
get_latest_files('tf_model_', X_train, Y_train)

In [ ]:
loaded_model = keras.models.load_model("model_6_model_0_chkpnt_.338-0.007420.hdf5")
output_ =loaded_model.predict(X_train)
for real, pred in zip(Y_train, output_):
    print(real[0], pred[0])

In [ ]:
reduce_lr = ReduceLROnPlateau(monitor='val_mae', factor=0.5,
                              patience=150, min_lr=0.00001, verbose = 1)
csv_logger = CSVLogger('keff_long_training.log')

checkpoint_filepath = ''
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_mae',
    mode='min',
    save_best_only=True,
    verbose = 1)


In [ ]:
loaded_model.fit(X_train,
                 Y_train,
                 validation_data=(X_test, Y_test),
                 epochs=3000,
                 batch_size=1000,
                 verbose=1,
                 callbacks=[reduce_lr, csv_logger, model_checkpoint_callback])

In [ ]:
LOG_DIR = "bayes_test_rep_gen_46_4_100_epoch_"
max_trials_val = 100
# int(max_trials_val/10)
tuner_bo = BayesianOptimization(
            hypermodel,
            objective='val_mae',
            max_trials=max_trials_val,
            seed=1,
            executions_per_trial=2,
            num_initial_points = int(max_trials_val/10),
            directory = LOG_DIR)
tuner_bo.search(x=X_train, y=Y_train,
             epochs=100,
             batch_size=2000,
             validation_data=(X_test, Y_test))

In [ ]:
tuner.results_summary()

In [ ]:
best_model = tuner.get_best_models(num_models=1)

In [ ]:
best_model[0].summary()

In [ ]:
best_model[0].save('best_model_2_1_3ed_fns_5_mat')

In [ ]:
loaded_model = keras.models.load_model("best_model_2_1_3ed_fns_5_mat")

In [ ]:
loaded_model.fit(X_train, Y_train, validation_data=(X_test, Y_test), epochs=1000, batch_size=2500, verbose=1)

In [ ]:
test_pred = loaded_model.predict(X_test)

In [ ]:
outputfile = open("out_2.csv", "w")
for _ in test_pred:
    str_ =str(_[0])+"\n"
    outputfile.write(str_)
outputfile.close()